In [11]:
from src.models.train_model import BuildModel 
from src.data.make_dataset import DataIngestion
from src.utils.common import read_yaml
from src import logger
from notebooks import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from pathlib import Path
import pandas as pd
import mlflow

from dataclasses import dataclass



In [19]:
read_yaml(CONFIG_FILE_PATH).data_directory.processed_test

2025-01-09 10:31:37,665 - STAGE 2: MODEL BUILDING - INFO - Yaml read successfully from /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/config/config.yaml


'data/processed/test/test.csv'

2025-01-09 10:33:18,277 - STAGE 2: MODEL BUILDING - INFO - Yaml read successfully from /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/config/config.yaml


PosixPath('/Users/goldyrana/mess/deep_learning/projects/telco_customer_churn')

In [18]:
Path(read_yaml(CONFIG_FILE_PATH).data_directory.processed_test).resolve().parent.parent

2025-01-09 10:31:24,985 - STAGE 2: MODEL BUILDING - INFO - Yaml read successfully from /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/config/config.yaml


PosixPath('/Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/notebooks/data/processed')

In [26]:
Path("data/processed/test/test.csv").resolve()

PosixPath('/Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/notebooks/data/processed/test/test.csv')

In [28]:
CONFIG_FILE_PATH

'/Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/config/config.yaml'

In [29]:
PARAMS_FILE_PATH

'/Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/params.yaml'

In [33]:
read_yaml(CONFIG_FILE_PATH).data_directory.processed_test

2025-01-09 10:36:30,940 - STAGE 2: MODEL BUILDING - INFO - Yaml read successfully from /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/config/config.yaml


'data/processed/test/test.csv'

In [35]:
@dataclass
class ModelEvaluationConfig:
    saved_model_dir: str
    test_dir: str
    metrics: list
    target_col: str

class ConfigurationManger:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

    def get_evaluation_config(self)-> ModelEvaluationConfig:
        model_evaluator_config = ModelEvaluationConfig(
        saved_model_dir=self.config.model.saved_model,
        test_dir=str(Path(self.config.data_directory.processed_test).resolve()),
        metrics=self.params.model.metrics,
        target_col=self.params.data.target_col)
        return model_evaluator_config
        

class ModelEvaluation(BuildModel):
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def load_model(self):
        try:
            model = mlflow.sklearn.load_model(self.config.saved_model_dir)
            logger.info("Model load sucessfully")
            return model
        except FileNotFoundError as e:
            logger.error(e)
        except Exception as e:
            logger.error(e)
    
    def evaluate(self, model, x, y_true):
        try:
            prediction = model.predict(x)
            self.log_metrics(model, self.config.metrics, y_true, prediction, "Test")
        except Exception as e:
            logger.error(e)
            raise e
        

if __name__ == "__main__":
    config_manager = ConfigurationManger()
    config_params = config_manager.get_evaluation_config()
    model_evaluator = ModelEvaluation(config_params)
    data_ingestion = DataIngestion()

    test = data_ingestion.get_data("/Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/data/processed/test/test.csv")
    x_test, y_test = data_ingestion.split_x_y(test, config_params.target_col)

    model = model_evaluator.load_model()
    model_evaluator.evaluate(model, x_test, y_test)



        
        





2025-01-09 10:37:14,336 - STAGE 2: MODEL BUILDING - INFO - Yaml read successfully from /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/config/config.yaml
2025-01-09 10:37:14,340 - STAGE 2: MODEL BUILDING - INFO - Yaml read successfully from /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/params.yaml
2025-01-09 10:37:14,347 - STAGE 2: MODEL BUILDING - INFO - Data read sucessfully /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/data/processed/test/test.csv
2025-01-09 10:37:14,348 - STAGE 2: MODEL BUILDING - INFO - Splitted data into x and y sucessfully
2025-01-09 10:37:14,351 - STAGE 2: MODEL BUILDING - ERROR - Could not find an "MLmodel" configuration file at "/Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/models/model.pkl"


TypeError: evaluate() takes 4 positional arguments but 5 were given